In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
MODEL_ID = os.getenv("BASE_MODEL_ID", "codellama/CodeLlama-7b-Instruct-hf")
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "./output")
DATA_PATH = os.getenv("DATA_PATH", "train.jsonl")

# Load dataset from jsonl
print("📦 Loading dataset...")
dataset = load_dataset("json", data_files=DATA_PATH, split="train")

📦 Loading dataset...


FileNotFoundError: Unable to find '/Users/keshavsaraogi/code/eureka/eureka_ivl/model/./train.jsonl'

In [ ]:
# Load tokenizer
print("🧠 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Prompt formatting
def format_prompt(example):
    prompt_text = f"<|user|>\n{example['prompt']}\n<|end|>\n<|assistant|>\n{example['response']}"
    tokenized = tokenizer(prompt_text, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔁 Formatting dataset...")
tokenized_dataset = dataset.map(format_prompt, remove_columns=["prompt", "response"])

In [ ]:
# Load model and apply LoRA
print("🔧 Loading model and applying LoRA...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_8bit=True
)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05
)

model = get_peft_model(model, lora_config)

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    fp16=True
)

In [ ]:
print("🚀 Starting training...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

In [ ]:
print("💾 Saving fine-tuned model...")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Done!")
